In [ ]:
!pip install sympy

## Fehlerrechnung
* automatische Ableitung der Funktion
* Einsetzen der Werte
* gibt Latex aus

Latex muss installiert sein 

In [ ]:
from sympy import *
from IPython.display import display
init_printing()
import numpy as np
from sympy.parsing.sympy_parser import parse_expr
from decimal import Decimal

class Parser:
    def __init__(self):
        self.reset()

    def reset(self):
        self.eq = None
        self.lhs = None
        self.rhs = None
        self.values = {}
        self.unit = None

    def geteq(self): 
        self.lhs, self.rhs = map(sympify, input("Equation: ").split("="))
        self.eq = Eq(self.lhs, self.rhs)
        unit = input("Unit: ")
        if unit == "%":
            self.unit = Symbol("\%")
        else:
            self.unit = sympify(unit, evaluate=False)
        return self.eq
    
    def getvals(self): 
        for sym in self.rhs.free_symbols:
            self.values[str(sym)] = Decimal(input(f"{sym}: "))
            self.values[f"\\Delta {sym}"] = Decimal(input(f"Δ{sym}: "))

    def wraplatex(self, obj): 
        w = "$$"
        if isinstance(obj, Basic):
            return f"{w}{latex(obj)}{w}".replace(".", "{,}").replace("Δ", "\\Delta ").replace("α", "\\alpha")
        return f"{w}{obj}{w}".replace(".", "{,}").replace("Δ", "\\Delta ").replace("α", "\\alpha")

    def eingeschissen(self, nachkommastellen=4):
        change = input("1. Change equation [f]\n2. Change values [v]\n(default=v)") or "v"
        if change == 'v':
            if not self.lhs: 
                print("Please define equation first!")
                self.geteq()
            display(self.eq)
            self.getvals()
        if change == 'f':
            self.geteq()
            display(self.eq)
            if sorted(list(self.values.keys())) != sorted([str(x) for x in self.rhs.free_symbols]):
                print("Please redefine the parameters")
                self.getvals()


        grosstfehler = 0
        veqs = []
        for sym in self.rhs.free_symbols:
            v = self.values[str(sym)]
            dv = self.values[f"\\Delta {sym}"]
            if dv != 0:
                veqs.append(Eq(sym, Symbol(f"({v} \pm {dv})")))
                # display(veq)
                # print(self.wraplatex(veq))
                grosstfehler = grosstfehler + (abs(self.rhs.diff(sym)) * Symbol(f'\Delta {sym}'))
            else:
                veqs.append(Eq(sym, v))
                # print(self.wraplatex())
        for veq in veqs: 
            display(veq)
        for veq in veqs: 
            print(self.wraplatex(veq))

        grosstfehler = simplify(grosstfehler)
        grosstfehler_result = round(lambdify(list(grosstfehler.free_symbols), grosstfehler, 'numpy')(*[self.values[str(x)] for x in grosstfehler.free_symbols]), nachkommastellen)

        result = round(lambdify(list(self.rhs.free_symbols), self.rhs, 'numpy')(*[self.values[str(x)] for x in self.rhs.free_symbols]), nachkommastellen)
        rel = round(grosstfehler_result/result*100, 2)
        grossfehlereq = Eq(Symbol(f'\Delta {self.lhs}'), grosstfehler)

        display(grossfehlereq)
        print(self.wraplatex(grossfehlereq), "\n")

        display(self.eq)
        print(self.wraplatex(self.eq), "\n")

        # display(Eq(Symbol(f'\Delta {self.lhs}'), deltaresult))
        # print(wraplatex+latex(Eq(Symbol(f'\Delta {self.f}'), deltaresult)).replace(".","{,}")+latex(self.unit)+wraplatex)
        # print()

        resulteq = Eq(self.lhs, Symbol(f'({result} \pm {grosstfehler_result})'))
        display(resulteq)
        print(self.wraplatex(resulteq), "\n")

        relfehlereq = Eq(Symbol(f'\Delta {self.lhs}')/self.lhs, Symbol(f'{rel}\%'))
        display(relfehlereq)
        print(self.wraplatex(relfehlereq), "\n")

        return
    
parser = Parser()

In [ ]:
parser.eingeschissen(nachkommastellen=16)

"""
Α α, Β β, Γ γ, Δ δ, Ε ε, Ζ ζ, Η η, Θ θ, Ι ι, Κ κ, Λ λ, Μ μ, 
Ν ν, Ξ ξ, Ο ο, Π π, Ρ ρ, Σ σ/ς, Τ τ, Υ υ, Φ φ, Χ χ, Ψ ψ, Ω ω
"""

"""
α=(ΔT*M_S*(m_e+m_L-c*m_L))/(K*c*m_L)-1
m_L: 40.7
Δm_L: 0.2
m_e: 1.5
Δm_e: 0.3
M_S: 58.443
ΔM_S: 0
c: 0.1
Δc: 0
ΔT: 1.5
ΔΔT: 0.2
K: 511.9
ΔK: 0.27
"""





## Steigungsfehler einer linearen Funktion
Es geschieht über lineare Regression der Fehlermaxima. Die Fehlergrenzgeraden liegen auf Flächen die von den Einzelfehlern von X und Y aufgespannt werden. Funktioniert nur wenn die Messwerte keine zu großen Messfehler aufweisen.

In [ ]:
def steigungsfehler(X, Y, Dx, Dy, plot=True, a=0.75):
    import numpy as np 
    X = np.array(X)
    Y = np.array(Y)
    n = len(X)
    lowerX = np.array([X[i] + 2*Dx*(i/(n-1))-Dx for i in range(n)])
    lowerY = np.array([Y[i] + 2*Dy*(i/(n-1))-Dy for i in range(n)])
    upperX = np.array([X[i] + 2*Dx*(i/(n-1))-Dx for i in range(n)])
    upperY = np.array([Y[i] - 2*Dy*(i/(n-1))+Dy for i in range(n)])

    def linreg(x, y):
        A = np.vstack([x, np.ones(len(x))]).T
        m, c = np.linalg.lstsq(A, y, rcond=None)[0]
        return m, c
    m, c = linreg(X, Y)
    LineX = np.array(X)
    LineY = m*LineX+c
    
    lowM, lowC = linreg((a*lowerX+(1-a)*X), (a*lowerY+(1-a)*Y))
    LineLowerX = np.array(lowerX)
    LineLowerY = lowM*LineLowerX+lowC

    upM, upC = linreg((a*upperX+(1-a)*X), (a*upperY+(1-a)*Y))
    LineUpperX = np.array(upperX)
    LineUpperY = upM*LineUpperX+upC

    if plot:
        plt.plot(LineX, LineY)
        plt.plot(LineLowerX, LineLowerY)
        plt.plot(LineUpperX, LineUpperY)

        plt.scatter(X, Y, marker='x')
        plt.scatter(lowerX, lowerY, marker='x')
        plt.scatter(upperX, upperY, marker='x')

        plt.show()
    return abs(upM - lowM) / 2

import matplotlib.pyplot as plt
X =[float(x) for x in input("X: ").strip().split(' ')]
Y = [float(x) for x in input("Y: ").strip().split(' ')]
DX = float(input("ΔX: "))
DY = float(input("ΔY: "))
steigungsfehler(X, Y, DX, DY)

In [ ]:
from sympy import sympify, Eq, latex
"""
Α α, Β β, Γ γ, Δ δ, Ε ε, Ζ ζ, Η η, Θ θ, Ι ι, Κ κ, Λ λ, Μ μ, 
Ν ν, Ξ ξ, Ο ο, Π π, Ρ ρ, Σ σ/ς, Τ τ, Υ υ, Φ φ, Χ χ, Ψ ψ, Ω ω
"""
rhs, lhs = map(sympify, "ΔT=m_B/(m_A*M_B)*K".split("="))
eq = Eq(rhs, lhs)
display(eq)
print(latex(eq, fold_func_brackets=True).replace("Δ", "\\Delta ").replace("α", "\\alpha").replace("Λ", "\\Lambda"))